In [1]:
import pandas as pd
import os
import re

In [8]:
def extract_temperature(filename):
    """
    Extract the numerical part of the temperature from the filename.
    Example: 'polishedYellow_30-3000cm-1_100%_532exc_15s_6K_point2' -> 6
    """
    match = re.search(r'(\d+)K', filename)
    return int(match.group(1)) if match else None

def extract_degree(filename):
    match = re.search(r'(\d+)_deg', filename)
    return int(match.group(1)) if match else None

def determine_test_type(filename):
    if 'cm-1' in filename.lower() and '30-2200cm-1' in filename:
        return 'raman'
    elif 'nm' in filename.lower() and '550-630nm' in filename:
        return 'pl'
    else:
        return 'unknown'


In [ ]:

# Your specified file path
directory = r'C:\Users\Q9gJYx\OneDrive - Duke University\Desktop\Mar_28_NaYb_Polar'

data_raman = {}
data_pl = {}

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        test_type = determine_test_type(filename)
        if test_type == 'unknown':
            continue

        temperature = extract_temperature(filename)
        file_path = os.path.join(directory, filename)

        data = pd.read_csv(file_path, sep='\t', header=None, names=['X', 'Y'])
        data.set_index('X', inplace=True)

        if test_type == 'raman':
            if temperature not in data_raman:
                data_raman[temperature] = []
            data_raman[temperature].append(data['Y'])
        elif test_type == 'pl':
            if temperature not in data_pl:
                data_pl[temperature] = []
            data_pl[temperature].append(data['Y'])

# Combine and average data with the same temperature
combined_data_raman = pd.DataFrame()
combined_data_pl = pd.DataFrame()

for temp, y_values in data_raman.items():
    combined_data_raman[temp] = pd.concat(y_values, axis=1).mean(axis=1)

for temp, y_values in data_pl.items():
    combined_data_pl[temp] = pd.concat(y_values, axis=1).mean(axis=1)

# Convert all column names to strings and then check if they are digits
combined_data_raman.columns = [int(str(col)) if str(col).isdigit() else col for col in combined_data_raman.columns]
combined_data_pl.columns = [int(str(col)) if str(col).isdigit() else col for col in combined_data_pl.columns]

# Sort the DataFrames by column names (which are now temperatures)
combined_data_raman = combined_data_raman.reindex(sorted(combined_data_raman.columns), axis=1)
combined_data_pl = combined_data_pl.reindex(sorted(combined_data_pl.columns), axis=1)

print("Sorted Raman Test Data:")
print(combined_data_raman)
print("\nSorted PL Test Data:")
print(combined_data_pl)

# After processing and printing the data
# Specify the Excel file name
excel_file_name = 'test_data_output1.xlsx'

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
    # Write each DataFrame to a different worksheet
    combined_data_raman.to_excel(writer, sheet_name='Raman Test Data')
    combined_data_pl.to_excel(writer, sheet_name='PL Test Data')

print(f"Data exported to {excel_file_name}")

In [9]:

# Your specified file path
directory = r'C:\Users\Q9gJYx\OneDrive - Duke University\Desktop\Mar_28_NaYb_Polar'

data_raman = {}
data_pl = {}

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        
        degree = extract_degree(filename)
        file_path = os.path.join(directory, filename)

        data = pd.read_csv(file_path, sep='\t', header=None, names=['X', 'Y'])
        data.set_index('X', inplace=True)
        
        if degree not in data_raman:
            data_raman[degree] = []
        data_raman[degree].append(data['Y'])

# Combine and average data with the same temperature
combined_data_raman = pd.DataFrame()

for degree, y_values in data_raman.items():
    combined_data_raman[degree] = pd.concat(y_values, axis=1).mean(axis=1)

# Convert all column names to strings and then check if they are digits
combined_data_raman.columns = [int(str(col)) if str(col).isdigit() else col for col in combined_data_raman.columns]

# Sort the DataFrames by column names (which are now temperatures)
combined_data_raman = combined_data_raman.reindex(sorted(combined_data_raman.columns), axis=1)

print("Sorted Raman Test Data:")
print(combined_data_raman)

# After processing and printing the data
# Specify the Excel file name
excel_file_name = 'test_data_output1.xlsx'

# Create a Pandas Excel writer using openpyxl as the engine
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
    # Write each DataFrame to a different worksheet
    combined_data_raman.to_excel(writer, sheet_name='Raman Test Data')

print(f"Data exported to {excel_file_name}")

Sorted Raman Test Data:
               0       20      40      60      80      100     120     140  \
X                                                                            
30.1612     832.75   781.0   717.0   716.5   761.0   966.0  1059.0  1179.5   
30.6877    1103.50  1003.0   932.0   924.0   998.0  1259.5  1393.0  1563.5   
31.2163    1418.00  1272.0  1157.5  1165.0  1287.0  1668.5  1832.5  1985.5   
31.7428    1848.75  1637.0  1502.5  1539.0  1720.0  2175.5  2399.0  2596.5   
32.2692    2399.00  2118.0  1932.0  2003.0  2249.0  2913.0  3155.5  3434.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
1998.3600   131.25   119.5   107.5    92.0   111.0   109.0    95.0   399.0   
1998.7600   133.25   118.0   102.0    88.0   107.0   106.0    96.0   409.0   
1999.1600   135.50   117.0   100.0    87.0   117.5   104.0   100.5   403.5   
1999.5600   129.25   115.5    88.0    90.0   112.5   106.0    99.5   416.5   
1999.9600   133.00   123.5    98.5    88